In [13]:
import os
os.getcwd()

'/Users/baeyoonseo/Desktop/portfolio/ChopChop/chopchop-yoonseo-analytics/notebooks'

In [14]:
import os

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
os.chdir(PROJECT_ROOT)

print("Current working directory:", os.getcwd())

Current working directory: /Users/baeyoonseo/Desktop/portfolio/ChopChop/chopchop-yoonseo-analytics


In [15]:
import numpy as np
import pandas as pd
from datetime import timedelta

In [16]:
np.random.seed(42)

N_USERS = 500
MAX_EVENTS_PER_USER = 50

start_date = pd.to_datetime("2025-01-01")
end_date = pd.to_datetime("2025-03-31")

In [17]:
users = pd.DataFrame({
    "user_id": range(1, N_USERS + 1),
    "signup_date": np.random.choice(
        pd.date_range(start_date, end_date), 
        size=N_USERS
    )
})

In [18]:
event_types = [
    "login",
    "view_recipe",
    "use_choptrack",
    "use_chopguide",
    "save_recipe",
    "logout"
]

def generate_user_events(user):
    n_events = np.random.randint(5, MAX_EVENTS_PER_USER)
    timestamps = sorted(
        np.random.choice(
            pd.date_range(user.signup_date, end_date, freq="H"),
            size=n_events
        )
    )
    
    events = np.random.choice(
        event_types,
        size=n_events,
        p=[0.25, 0.25, 0.2, 0.15, 0.1, 0.05]
    )
    
    return pd.DataFrame({
        "user_id": user.user_id,
        "event_time": timestamps,
        "event_type": events
    })


In [19]:
event_logs = pd.concat(
    users.apply(generate_user_events, axis=1).tolist(),
    ignore_index=True
)

event_logs.head()


/var/folders/8g/zv0955h54s17sjwt7_k9btb40000gn/T/ipykernel_38745/2777746616.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pd.date_range(user.signup_date, end_date, freq="H"),


,user_id,event_time,event_type
0,1,2025-02-21 11:00:00,view_recipe
1,1,2025-02-22 10:00:00,use_choptrack
2,1,2025-02-24 17:00:00,view_recipe
3,1,2025-02-25 18:00:00,save_recipe
4,1,2025-03-03 12:00:00,login


In [20]:
event_logs["used_choptrack"] = event_logs["event_type"] == "use_choptrack"
event_logs["used_chopguide"] = event_logs["event_type"] == "use_chopguide"


In [21]:
first_events = (
    event_logs.groupby("user_id")["event_time"]
    .min()
    .reset_index(name="first_event_time")
)

event_logs = event_logs.merge(first_events, on="user_id")

event_logs["days_since_first"] = (
    event_logs["event_time"] - event_logs["first_event_time"]
).dt.days

event_logs["retained_7d"] = event_logs["days_since_first"] >= 7


In [22]:
event_logs.to_csv("data/raw/events_raw.csv", index=False)

print("data/raw/events_raw.csv saved")


data/raw/events_raw.csv saved
